In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
file_path = './data/f1_data_2018_2021p.csv'
data_df = pd.read_csv(file_path)

In [5]:
data_df.shape

(1360, 37)

In [8]:
columns_to_drop = ['Q1_Race', 'Q2_Race', 'Q3_Race', 'CountryCode', 'FirstName', 'LastName', 
                   'DriverNumber', 'BroadcastName', 'Abbreviation', 'Points', 'HeadshotUrl', 'TeamColor', 
                   'TeamName', 'FullName']

In [9]:
data_df = data_df.drop(columns=columns_to_drop)

In [31]:
data_df.columns

Index(['DriverId', 'TeamId', 'Position_Race', 'ClassifiedPosition',
       'GridPosition', 'Time', 'Status', 'RaceDate', 'TotalLaps', 'LapLength',
       'Position_Qual', 'Q1_Qual', 'Q2_Qual', 'Q3_Qual', 'AirTemp', 'Humidity',
       'Pressure', 'Rainfall', 'TrackTemp', 'WindDirection', 'WindSpeed',
       'Year', 'RaceName'],
      dtype='object')

In [32]:
data_df.head(5)

,DriverId,TeamId,Position_Race,ClassifiedPosition,GridPosition,Time,Status,RaceDate,TotalLaps,LapLength,...,Q3_Qual,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Year,RaceName
0,vettel,ferrari,1,1,3,0 days 01:29:33.283000,Finished,3/25/2018 5:10,58,5278,...,0 days 00:01:21.838000,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
1,hamilton,mercedes,2,2,1,0 days 01:29:38.319000,Finished,3/25/2018 5:10,58,5278,...,0 days 00:01:21.164000,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
2,raikkonen,ferrari,3,3,2,0 days 01:29:39.592000,Finished,3/25/2018 5:10,58,5278,...,0 days 00:01:21.828000,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
3,ricciardo,red_bull,4,4,8,0 days 01:29:40.352000,Finished,3/25/2018 5:10,58,5278,...,0 days 00:01:22.152000,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
4,alonso,mclaren,5,5,10,0 days 01:30:01.169000,Finished,3/25/2018 5:10,58,5278,...,NaN,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix


In [45]:
data_df.isnull().sum()

DriverId                0
TeamId                  0
Position_Race           0
ClassifiedPosition      0
GridPosition            0
Time                  696
Status                  0
RaceDate                0
Position_Qual           0
Q1_Qual                22
Q2_Qual               365
Q3_Qual               698
AirTemp                 0
Humidity                0
Pressure                0
Rainfall                0
TrackTemp               0
WindDirection           0
WindSpeed               0
Year                    0
RaceName                0
TotalLength             0
dtype: int64

In [37]:
# change Time into timedelta object, and then convert to total seconds
data_df['Time'] = pd.to_timedelta(data_df['Time'], errors='coerce').dt.total_seconds()

In [36]:
# calculate the total length of circuit, and then drop the total laps and lap length
data_df['TotalLength'] = data_df['TotalLaps'] * data_df['LapLength']
data_df.drop(columns=['TotalLaps', 'LapLength'], inplace=True)

In [48]:
# create a binary indicator for if the driver finished the race successfully or not
data_df['Finished'] = data_df['ClassifiedPosition'].apply(lambda x: 0 if x in ['R', 'W', 'D'] else 1)

In [46]:
# Fill Position_Qual null values with 20 (ranked last)
data_df['Position_Qual'] = data_df['Position_Qual'].fillna(20)
# Replace 0s in GridPosition with 20, checked manually
data_df['GridPosition'] = data_df['GridPosition'].replace(0, 20)

In [49]:
data_df.columns

Index(['DriverId', 'TeamId', 'Position_Race', 'ClassifiedPosition',
       'GridPosition', 'Time', 'Status', 'RaceDate', 'Position_Qual',
       'Q1_Qual', 'Q2_Qual', 'Q3_Qual', 'AirTemp', 'Humidity', 'Pressure',
       'Rainfall', 'TrackTemp', 'WindDirection', 'WindSpeed', 'Year',
       'RaceName', 'TotalLength', 'Finished'],
      dtype='object')

In [53]:
data_df.to_csv('./data/f1_data_processed.csv')

In [33]:
data_df['Status'].unique()

array(['Finished', '+1 Lap', 'Wheel', 'Engine', 'Steering', 'Brakes',
       'Puncture', 'Electrical', 'Collision damage', 'Accident',
       'Collision', '+2 Laps', '+3 Laps', 'Gearbox', 'Oil leak', 'Turbo',
       'Exhaust', 'Fuel pressure', 'Hydraulics', 'Tyre', 'Power Unit',
       'Retired', 'Power loss', 'Suspension', 'Disqualified',
       'Mechanical', 'Battery', 'Overheating', 'Damage', 'Out of fuel',
       'Transmission', 'Spun off', 'Water pressure', 'Withdrew',
       'Electronics', '+5 Laps', 'Debris', 'Radiator', 'Illness',
       'Rear wing', 'Wheel nut', 'Driveshaft'], dtype=object)

In [14]:
data_df[data_df['Position_Qual'].isna()].iloc[0]

DriverId                           albon
TeamId                        toro_rosso
Position_Race                         10
ClassifiedPosition                    10
GridPosition                           0
Time                                 NaN
Status                            +1 Lap
RaceDate                  4/14/2019 6:10
TotalLaps                             56
LapLength                           5451
Position_Qual                        NaN
Q1_Qual                              NaN
Q2_Qual                              NaN
Q3_Qual                              NaN
AirTemp                        19.329921
Humidity                       45.981102
Pressure                      1019.21811
Rainfall                             0.0
TrackTemp                      27.337008
WindDirection                   93.80315
WindSpeed                       3.315748
Year                                2019
RaceName              Chinese Grand Prix
Name: 469, dtype: object

In [42]:
data_df[data_df['GridPosition'] == 0][['GridPosition', 'Position_Qual']]

,GridPosition,Position_Qual
199,0,20.0
469,0,NaN
489,0,19.0
495,0,18.0
496,0,20.0
512,0,16.0
556,0,10.0
597,0,19.0
676,0,20.0
694,0,10.0


In [31]:
data_df[['Position_Race', 'ClassifiedPosition']].head(20)

,Position_Race,ClassifiedPosition
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9
9,10,10


In [30]:
data_df['ClassifiedPosition'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'R', '16', '17', '18', '19', '20', 'D', 'W'],
      dtype=object)

In [12]:
data_df[data_df['GridPosition'] != data_df['Position_Qual']]

,DriverId,TeamId,Position_Race,ClassifiedPosition,GridPosition,Time,Status,RaceDate,TotalLaps,LapLength,...,Q3_Qual,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Year,RaceName
3,ricciardo,red_bull,4,4,8,0 days 01:29:40.352000,Finished,3/25/2018 5:10,58,5278,...,0 days 00:01:22.152000,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
4,alonso,mclaren,5,5,10,0 days 01:30:01.169000,Finished,3/25/2018 5:10,58,5278,...,NaN,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
6,hulkenberg,renault,7,7,7,0 days 01:30:05.954000,Finished,3/25/2018 5:10,58,5278,...,0 days 00:01:23.532000,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
7,bottas,mercedes,8,8,15,0 days 01:30:07.622000,Finished,3/25/2018 5:10,58,5278,...,NaN,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
8,vandoorne,mclaren,9,9,11,0 days 01:30:08.204000,Finished,3/25/2018 5:10,58,5278,...,NaN,24.077477,30.915315,997.003604,0.045045,36.324324,294.126126,3.691892,2018,Australian Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,mick_schumacher,haas,18,18,20,NaN,+3 Laps,5/23/2021 13:00,78,3337,...,NaN,20.760741,61.424444,1015.571111,0.000000,37.595556,202.081481,0.316296,2021,Monaco Grand Prix
1340,perez,red_bull,1,1,6,0 days 02:13:36.410000,Finished,6/6/2021 12:00,51,6003,...,0 days 00:01:41.917000,24.170930,55.805233,1014.730814,0.000000,38.314535,180.186047,0.283721,2021,Azerbaijan Grand Prix
1344,norris,mclaren,5,5,9,0 days 02:13:41.164000,Finished,6/6/2021 12:00,51,6003,...,0 days 00:01:41.747000,24.170930,55.805233,1014.730814,0.000000,38.314535,180.186047,0.283721,2021,Azerbaijan Grand Prix
1345,alonso,alpine,6,6,8,0 days 02:13:42.792000,Finished,6/6/2021 12:00,51,6003,...,0 days 00:01:42.327000,24.170930,55.805233,1014.730814,0.000000,38.314535,180.186047,0.283721,2021,Azerbaijan Grand Prix


In [30]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360 entries, 0 to 1359
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DriverId            1360 non-null   object 
 1   TeamId              1360 non-null   object 
 2   Position_Race       1360 non-null   int64  
 3   ClassifiedPosition  1360 non-null   object 
 4   GridPosition        1360 non-null   int64  
 5   Time                664 non-null    object 
 6   Status              1360 non-null   object 
 7   RaceDate            1360 non-null   object 
 8   TotalLaps           1360 non-null   int64  
 9   LapLength           1360 non-null   int64  
 10  Position_Qual       1357 non-null   float64
 11  Q1_Qual             1338 non-null   object 
 12  Q2_Qual             995 non-null    object 
 13  Q3_Qual             662 non-null    object 
 14  AirTemp             1360 non-null   float64
 15  Humidity            1360 non-null   float64
 16  Pressu